# Building an Image Classifier Deep Network applying Transfer Learning

In [12]:
import os

# mount_path = os.environ['working_dir']

In [13]:
# import the required libraries
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# from tensorflow.keras.preprocessing import image_dataset_from_directory
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import tf.keras.utils
from PIL import Image
import numpy as np
import os
import base64
from io import BytesIO

# Import pre-trained model

In [14]:
pre_trained_model = tf.keras.applications.MobileNetV2()
len(pre_trained_model.layers)

156

# Get Labels

In [15]:
LABELS_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt', LABELS_URL)
class_names = np.array(
    open(labels_path).read().splitlines()
)[1:]

print('Labels shape:', class_names.shape)

print(class_names)

Labels shape: (1000,)
['tench' 'goldfish' 'great white shark' 'tiger shark' 'hammerhead'
 'electric ray' 'stingray' 'cock' 'hen' 'ostrich' 'brambling' 'goldfinch'
 'house finch' 'junco' 'indigo bunting' 'robin' 'bulbul' 'jay' 'magpie'
 'chickadee' 'water ouzel' 'kite' 'bald eagle' 'vulture' 'great grey owl'
 'European fire salamander' 'common newt' 'eft' 'spotted salamander'
 'axolotl' 'bullfrog' 'tree frog' 'tailed frog' 'loggerhead'
 'leatherback turtle' 'mud turtle' 'terrapin' 'box turtle' 'banded gecko'
 'common iguana' 'American chameleon' 'whiptail' 'agama' 'frilled lizard'
 'alligator lizard' 'Gila monster' 'green lizard' 'African chameleon'
 'Komodo dragon' 'African crocodile' 'American alligator' 'triceratops'
 'thunder snake' 'ringneck snake' 'hognose snake' 'green snake'
 'king snake' 'garter snake' 'water snake' 'vine snake' 'night snake'
 'boa constrictor' 'rock python' 'Indian cobra' 'green mamba' 'sea snake'
 'horned viper' 'diamondback' 'sidewinder' 'trilobite' 'harvest

# test single image

In [16]:
testfile = "samples/banana.jpeg"

img = Image.open(testfile).convert('RGB')
img = img.resize((224, 224), Image.LANCZOS)
img = np.asarray(img)

mydata = np.empty((1, 224, 224, 3))
mydata[0] = img

ppdata = preprocess_input(mydata)

mypredition = pre_trained_model.predict(ppdata)

print(class_names[np.argmax(mypredition, axis=1)])

1/1 [==============================] - 1s 636ms/step
['banana']


# Save Base model

In [17]:
# We save the model in order to genereate a default signature we can later modify
tf.saved_model.save(pre_trained_model, "./models/base/1")


INFO:tensorflow:Assets written to: ./models/base/1/assets


INFO:tensorflow:Assets written to: ./models/base/1/assets


# Save model with Base64 Signature

### Load labels

In [18]:
print(class_names)
labels = tf.constant([class_names])
# print(labels)

['tench' 'goldfish' 'great white shark' 'tiger shark' 'hammerhead'
 'electric ray' 'stingray' 'cock' 'hen' 'ostrich' 'brambling' 'goldfinch'
 'house finch' 'junco' 'indigo bunting' 'robin' 'bulbul' 'jay' 'magpie'
 'chickadee' 'water ouzel' 'kite' 'bald eagle' 'vulture' 'great grey owl'
 'European fire salamander' 'common newt' 'eft' 'spotted salamander'
 'axolotl' 'bullfrog' 'tree frog' 'tailed frog' 'loggerhead'
 'leatherback turtle' 'mud turtle' 'terrapin' 'box turtle' 'banded gecko'
 'common iguana' 'American chameleon' 'whiptail' 'agama' 'frilled lizard'
 'alligator lizard' 'Gila monster' 'green lizard' 'African chameleon'
 'Komodo dragon' 'African crocodile' 'American alligator' 'triceratops'
 'thunder snake' 'ringneck snake' 'hognose snake' 'green snake'
 'king snake' 'garter snake' 'water snake' 'vine snake' 'night snake'
 'boa constrictor' 'rock python' 'Indian cobra' 'green mamba' 'sea snake'
 'horned viper' 'diamondback' 'sidewinder' 'trilobite' 'harvestman'
 'scorpion' 'blac

### Load base model, add signature and save

In [19]:
smodel = tf.saved_model.load("./models/base/1")

# This is the current signature, that only accepts image tensors as input
signature = smodel.signatures["serving_default"]
print(signature)

# obtain the key name of the output (typically 'dense_X')
keyOutput = next(iter(signature.structured_outputs))

@tf.function()
def my_predict(image_b64):

    # get content
    content = image_b64[0]
    
    # decode image    
    image = tf.image.decode_jpeg(content,channels=3)
    # tf.compat.v1.enable_eager_execution()
    
    # resize image
    # image = tf.image.resize(image, [160, 160])
    image = tf.image.resize(image, [224, 224])
    
    # expand dimension to match signature
    image = tf.expand_dims(image, 0)
    
    image = preprocess_input(image)
        
    # execute prediction
    prediction = signature(image)

    # obtain index of maximum probability prediction
    idx = tf.argmax(prediction[keyOutput],axis=1)
    
    # obtain the label for given index
    label = tf.gather(labels, idx, batch_dims=1)

    # obtain probability from Tensor
    probability = prediction[keyOutput][0,idx[0]]
    
    # combine result in String Tensor format with [label,probability]
    result = tf.concat([label, [tf.as_string(probability)]], axis=0)
    
    # return result_tensor
    return result

# Create new signature, to read b64 images
new_signature = my_predict.get_concrete_function(
    image_b64=tf.TensorSpec(name="image_b64", shape=[1], dtype=tf.string)
)

# Save model with Base64 input signature
tf.saved_model.save(
    smodel,
    export_dir="./models/redbag/1",
    signatures=new_signature
)

ConcreteFunction signature_wrapper(*, input_2)
  Args:
    input_2: float32 Tensor, shape=(None, 224, 224, 3)
  Returns:
    {'predictions': <1>}
      <1>: float32 Tensor, shape=(None, 1000)
INFO:tensorflow:Assets written to: ./models/redbag/1/assets


INFO:tensorflow:Assets written to: ./models/redbag/1/assets


# Test Base64 Model with single image

In [20]:
smodel = tf.saved_model.load("./models/redbag/1")

# Load model's signature
signature = smodel.signatures["serving_default"]

print(signature)

ConcreteFunction signature_wrapper(*, image_b64)
  Args:
    image_b64: string Tensor, shape=(1,)
  Returns:
    {'output_0': <1>}
      <1>: string Tensor, shape=(2,)


### Run Inference

In [21]:
testfile = "./samples/banana.jpeg"

# load test image
content = tf.io.read_file(testfile)

# reshape to signature's expected dimensions
content = tf.reshape(content, shape = [1])

# print(tf.print(content, summarize=3))

# obtain signature
f = smodel.signatures["serving_default"]

# run prediction
myprediction = f(image_b64=content)
print(myprediction)

{'output_0': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'banana', b'0.808100'], dtype=object)>}
